# Locations of Neighborhoods in Toronto

In [11]:
from bs4 import BeautifulSoup
import requests
import csv
import json
import xml
import pandas as pd

In [12]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text

soup = BeautifulSoup(source, 'lxml')#Beautiful Soup to Parse the url page

table=soup.find('table') #Finds the required table area

#List initialization to collect the Postalcodes, Boroughs and Neighborhoods
postalcode=[]
borough=[]
neighborhood=[]

x=table.tbody #Navidation to the body of the table

for tr in x.find_all('tr'): #Iterating through rows and columns
    td=tr.find_all('td')
    for x in td:
        #Exception handling to bypass the errors
        try:
            if x.span.text!='Not assigned':
                postalcode.append(x.b.text) #Collecting the Postalcodes
        except:
            pass
        try:
            if x.span.text!='Not assigned': 
                my=x.span.get_text(separator=u' ').split(' ')
                
                if my[1]=='York':
                    my_b= my[0]+' '+my[1]
                    borough.append(my_b)
                    my_nei=my[2:]
                    if my_nei==['\n'] or my_nei==[]:
                        neighborhood.append(my_b)
                    else:
                        my_nei1=' '.join(my_nei)
                        neighborhood.append(my_nei1.replace('(',',',5).replace(')',',',5).replace('/',',',5).strip(', '))
                
                elif my[1]=='Toronto':
                    my_b=my[0]+' '+my[1]
                    borough.append(my_b)
                    my_nei=my[2:]
                    if my_nei==['\n'] or my_nei==[]:
                        neighborhood.append(my_b)
                    else:
                        my_nei1=' '.join(my_nei)
                        neighborhood.append(my_nei1.replace('(',',',5).replace(')',',',5).replace('/',',',5).strip(', '))
                
                elif my[1]=='Park':
                    my_b=my[0]+' '+my[1]
                    borough.append(my_b)
                    my_nei=my[2:]
                    if my_nei==['\n'] or my_nei==[]:
                        neighborhood.append(my_b)
                    else:
                        my_nei1=' '.join(my_nei)
                        neighborhood.append(my_nei1.replace('(',',',5).replace(')',',',5).replace('/',',',5).strip(', '))
                
                else:
                    my_b=my[0]
                    borough.append(my_b)
                    my_nei=my[2:]
                    if my_nei==['\n'] or my_nei==[]:
                        neighborhood.append(my_b)
                    else:
                        my_nei1=' '.join(my_nei)
                        neighborhood.append(my_nei1.replace('(',',',5).replace(')',',',5).replace('/',',',5).strip(', '))
       
    
         
        except:
            pass



In [13]:
#Constructing the main dataframe 
main_df=pd.DataFrame({'PostalCode':postalcode,
                 'Borough':borough,
                 'Neighborhood':neighborhood})

In [14]:
#Fixing the order of the columns in the main dataframe
main_df=main_df[['PostalCode','Borough','Neighborhood']]

In [15]:
#Visualizing the first few rows of the dataframe
main_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,"Don Mills , North"
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District , Ryerson"


In [16]:
# The code was removed by Watson Studio for sharing.

In [17]:
latitude=[] #List to collect the latitudes
longitude=[] #List to collect the longitudes

for i in main_df['PostalCode']: #Iterating through Postalcodes to collect the locations data
    try:
        j='toronto,'+i
        url ="https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}".format(API_key,j)
        response = requests.get(url).json() # get response
        geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
        latitude.append(geographical_data['lat'])
        longitude.append(geographical_data['lng'])
    except:
        pass
main_df['Latitude']=latitude #Adding a column in the main dataframe for Latitude  

main_df['Longitude']=longitude #Adding a column in the main dataframe for Longitude 
main_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [18]:
main_df.Borough.unique()


array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [19]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(main_df['Borough'].unique()),
      main_df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.
